In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

import copy

import rasterio as rio
import rasterio.windows

from tqdm.notebook import tqdm

In [ ]:
spectrum = []
with open('data2/spectra/1/splib07a_Oil_Water_Benzene17_DWH10-3_ASDHRa_AREF.txt', 'r') as f:
    f.readline()
    for line in f.readlines():
        spectrum.append(float(line))
spectrum = np.array(spectrum)
spectrum = spectrum / np.linalg.norm(spectrum, ord=2)
spectrum_normalized = scipy.signal.resample(spectrum, 224) - spectrum.mean()

In [ ]:
plt.plot(spectrum_normalized)

In [ ]:
def infer(in_filename, out_filename):
    with rio.open(in_filename, 'r') as in_ds:
        profile = copy.deepcopy(in_ds.profile)
        profile.update(count=1, driver='GTiff', bigtiff='yes', compress='deflate', predictor='2', tiled='yes', dtype=np.float32)
        with rio.open(out_filename, 'w', **profile) as out_ds:
            for col in tqdm(range(0, in_ds.width, 512), position=0):
                width = min(col+512, in_ds.width) - col
                for row in tqdm(range(0, in_ds.height, 512), position=1, leave=False):
                    height = min(row+512, in_ds.height) - row
                    window = rasterio.windows.Window(col, row, width, height)
                    data = np.transpose(in_ds.read(window=window).astype(np.float32), (1,2,0))
                    norm = np.linalg.norm(data, ord=2, axis=2)[..., None].astype(np.float32)
                    data /= norm
                    data -= np.mean(data, axis=2)[...,None]
                    data = np.dot(data, spectrum_normalized)
                    data = data.reshape(1, width, height).astype(np.float32)
                    out_ds.write(data, window=window)

In [ ]:
in_out = [
 ('data2/f100520t01p00r17rdn_b/f100520t01p00r17rdn_b_sc01_ort_img.tif', 'data/f100520t01p00r17rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100824t01p00r25rdn_b/f100824t01p00r25rdn_b_sc01_ort_img.tif', 'data/f100824t01p00r25rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100506t01p00r17rdn_b/f100506t01p00r17rdn_b_sc01_ort_img.tif', 'data/f100506t01p00r17rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100517t01p00r11rdn_b/f100517t01p00r11rdn_b_sc01_ort_img.tif', 'data/f100517t01p00r11rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100709t02p00r05rdn_b/f100709t02p00r05rdn_b_sc01_ort_img.tif', 'data/f100709t02p00r05rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100520t01p00r11rdn_b/f100520t01p00r11rdn_b_sc01_ort_img.tif', 'data/f100520t01p00r11rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100517t01p00r14rdn_b/f100517t01p00r14rdn_b_sc01_ort_img.tif', 'data/f100517t01p00r14rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100506t01p00r18rdn_b/f100506t01p00r18rdn_b_sc01_ort_img.tif', 'data/f100506t01p00r18rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100824t01p00r24rdn_b/f100824t01p00r24rdn_b_sc01_ort_img.tif', 'data/f100824t01p00r24rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100524t01p00r11rdn_b/f100524t01p00r11rdn_b_sc01_ort_img.tif', 'data/f100524t01p00r11rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100709t02p00r06rdn_b/f100709t02p00r06rdn_b_sc01_ort_img.tif', 'data/f100709t02p00r06rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100709t01p00r07rdn_b/f100709t01p00r07rdn_b_sc01_ort_img.tif', 'data/f100709t01p00r07rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100506t01p00r16rdn_b/f100506t01p00r16rdn_b_sc01_ort_img.tif', 'data/f100506t01p00r16rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100518t01p00r11rdn_b/f100518t01p00r11rdn_b_sc01_ort_img.tif', 'data/f100518t01p00r11rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100513t01p00r15rdn_b/f100513t01p00r15rdn_b_sc01_ort_img.tif', 'data/f100513t01p00r15rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100709t01p00r05rdn_b/f100709t01p00r05rdn_b_sc01_ort_img.tif', 'data/f100709t01p00r05rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100831t01p00r05rdn_b/f100831t01p00r05rdn_b_sc01_ort_img.tif', 'data/f100831t01p00r05rdn_b_sc01_ort_img_result.tif'),
 ('data2/f100510t01p00r16rdn_b/f100510t01p00r16rdn_b_sc01_ort_img.tif', 'data/f100510t01p00r16rdn_b_sc01_ort_img_result.tif')
]

In [ ]:
for (in_filename, out_filename) in in_out:
    infer(in_filename, out_filename)